<a href="https://colab.research.google.com/github/Dehan001/data_science/blob/main/linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up Necessary Things

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import tensor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MasterCourse/Week 11 - Building Neural Networks with PyTorch/Lecture 41 - Build A Linear Model

/content/drive/MyDrive/MasterCourse/Week 11 - Building Neural Networks with PyTorch/Lecture 41 - Build A Linear Model


# Get Data

The reason we converted categorical values into numbers because computer do not understand text.

In [ ]:
train_df = pd.read_csv("adult_train.csv")
test_df = pd.read_csv("adult_test.csv")
train_df.head()

,age,fnlwgt,educational-num,hours-per-week,log-capital-gain,log-capital-loss,private_workclass,postgrad_education,after_HS_education,school_education,...,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_white,gender_male,country_US,high_income
0,71,77253,9,17,0.0,0.0,True,0,1,0,...,0,1,0,0,0,0,True,True,True,False
1,17,329783,6,10,0.0,0.0,True,0,0,1,...,0,0,1,0,0,0,True,False,True,False
2,27,91257,9,40,0.0,0.0,True,0,1,0,...,1,0,0,0,0,0,True,True,False,False
3,43,125577,9,40,0.0,0.0,True,0,1,0,...,0,0,0,0,1,0,False,False,True,False
4,31,137978,13,40,0.0,0.0,True,0,1,0,...,1,0,0,0,0,0,True,True,True,False


In [ ]:
train_df.shape, test_df.shape

((39073, 25), (9769, 25))

In [ ]:
train_df.columns

Index(['age', 'fnlwgt', 'educational-num', 'hours-per-week',
       'log-capital-gain', 'log-capital-loss', 'private_workclass',
       'postgrad_education', 'after_HS_education', 'school_education',
       'married', 'occupation_cat_blue_collar', 'occupation_cat_others',
       'occupation_cat_service', 'occupation_cat_white_collar',
       'relationship_Husband', 'relationship_Not-in-family',
       'relationship_Other-relative', 'relationship_Own-child',
       'relationship_Unmarried', 'relationship_Wife', 'race_white',
       'gender_male', 'country_US', 'high_income'],
      dtype='object')

In [ ]:
labels = tensor(train_df.high_income, dtype=torch.int)
labels[:5]

tensor([0, 0, 0, 0, 0], dtype=torch.int32)

In [ ]:
feature_df = train_df[train_df.columns[:-1]].astype(float)
features = tensor(feature_df.values, dtype=torch.float)
features[:5]

tensor([[7.1000e+01, 7.7253e+04, 9.0000e+00, 1.7000e+01, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.7000e+01, 3.2978e+05, 6.0000e+00, 1.0000e+01, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00],
        [2.7000e+01, 9.1257e+04, 9.0000e+00, 4.0000e+01, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00],
        [4.3000e+01, 1.25

In [ ]:
features.shape, labels.shape

(torch.Size([39073, 24]), torch.Size([39073]))

# One Step of Linear Model

1. Initialize weights randomly and normalize data
2. Calculate Prediction (weights * features) { $h(x) = W * x$ }
3. Calculate Loss
4. Backpropagation
5. Gradient Step $W = W - lr * grad$

### Initialize weights randomly

In [ ]:
torch.manual_seed(42)
n_params = features.shape[1]
weights = torch.rand(n_params)
weights

tensor([0.8823, 0.9150, 0.3829, 0.9593, 0.3904, 0.6009, 0.2566, 0.7936, 0.9408,
        0.1332, 0.9346, 0.5936, 0.8694, 0.5677, 0.7411, 0.4294, 0.8854, 0.5739,
        0.2666, 0.6274, 0.2696, 0.4414, 0.2969, 0.8317])

In [ ]:
weights.shape

torch.Size([24])

### Normalization

In [ ]:
max_value, _ = features.max(dim=0)
features /= max_value
features

tensor([[0.7889, 0.0520, 0.5625,  ..., 1.0000, 1.0000, 1.0000],
        [0.1889, 0.2221, 0.3750,  ..., 1.0000, 0.0000, 1.0000],
        [0.3000, 0.0615, 0.5625,  ..., 1.0000, 1.0000, 0.0000],
        ...,
        [0.4000, 0.0920, 0.6250,  ..., 1.0000, 1.0000, 1.0000],
        [0.3556, 0.0539, 0.5625,  ..., 1.0000, 1.0000, 1.0000],
        [0.3333, 0.1355, 0.4375,  ..., 0.0000, 0.0000, 1.0000]])

### Calculate Prediction

In [ ]:
def calculate_preds(weights, features):
  mults = (features * weights)
  preds = mults.sum(axis=1)
  return preds

In [ ]:
preds = calculate_preds(weights, features)
preds[:5]

tensor([5.3700, 3.5882, 4.7912, 4.4995, 5.9600])

### Calculate Loss

In [ ]:
def calculate_loss(preds, labels):
  return torch.abs(preds-labels).mean()

In [ ]:
loss = calculate_loss(preds, labels)
loss

tensor(4.9282)

### Calculate Gradients and Backpropagation

In [ ]:
weights.requires_grad_()

tensor([0.8823, 0.9150, 0.3829, 0.9593, 0.3904, 0.6009, 0.2566, 0.7936, 0.9408,
        0.1332, 0.9346, 0.5936, 0.8694, 0.5677, 0.7411, 0.4294, 0.8854, 0.5739,
        0.2666, 0.6274, 0.2696, 0.4414, 0.2969, 0.8317], requires_grad=True)

In [ ]:
preds = calculate_preds(weights, features)
loss = calculate_loss(preds, labels)
loss

tensor(4.9282, grad_fn=<MeanBackward0>)

In [ ]:
# Backpropagation
loss.backward()

In [ ]:
weights.grad

tensor([0.4300, 0.1277, 0.6300, 0.4087, 0.0626, 0.0430, 0.6937, 0.0672, 0.8016,
        0.1312, 0.4703, 0.3276, 0.0575, 0.1059, 0.5090, 0.4026, 0.2586, 0.0314,
        0.1549, 0.1052, 0.0473, 0.8561, 0.6690, 0.8983])

### Gradient Step

In [ ]:
learning_rate = 0.1

with torch.no_grad():
  weights.sub_(weights.grad * learning_rate)
  preds = calculate_preds(weights, features)
  loss = calculate_loss(preds, labels)
  print(loss)

tensor(4.4509)


# Setting Up everything together

In [ ]:
train_feature_df = train_df[train_df.columns[:-1]].astype(float)
train_features = tensor(train_feature_df.values, dtype=torch.float)
train_labels = tensor(train_df.high_income, dtype=torch.int)

In [ ]:
max_value, _ = train_features.max(dim=0)
train_features /= max_value

In [ ]:
test_feature_df = test_df[test_df.columns[:-1]].astype(float)
test_features = tensor(test_feature_df.values, dtype=torch.float)
test_labels = tensor(test_df.high_income, dtype=torch.int)

In [ ]:
max_value, _ = test_features.max(dim=0)
test_features /= max_value

In [ ]:
def init_weights():
  torch.manual_seed(42)
  n_params = train_features.shape[1]
  weights = torch.rand(n_params)
  return weights.requires_grad_()

In [ ]:
def calculate_preds(weights, features):
  mults = (features * weights)
  preds = mults.sum(axis=1)
  return preds

In [ ]:
def calculate_loss(preds, labels):
  return torch.abs(preds-labels).mean()

In [ ]:
def update_weights(weights, lr):
  weights.sub_(weights.grad * lr)
  return

In [ ]:
def one_epoch(epoch_no, weights, lr):
  preds = calculate_preds(weights, features)
  loss = calculate_loss(preds, train_labels)
  loss.backward()
  with torch.no_grad(): update_weights(weights, lr)
  print(f"Epoch {epoch_no+1} => Loss: {loss}")

In [ ]:
def train_model(epochs, lr):
  weights = init_weights()
  for epoch_no in range(epochs):
    one_epoch(epoch_no, weights, lr)
  return weights

In [ ]:
weights = train_model(epochs=15, lr=0.01)

Epoch 1 => Loss: 4.928189754486084
Epoch 2 => Loss: 4.880458354949951
Epoch 3 => Loss: 4.7849955558776855
Epoch 4 => Loss: 4.641801834106445
Epoch 5 => Loss: 4.450876712799072
Epoch 6 => Loss: 4.212220668792725
Epoch 7 => Loss: 3.9258322715759277
Epoch 8 => Loss: 3.5917136669158936
Epoch 9 => Loss: 3.2098629474639893
Epoch 10 => Loss: 2.7802815437316895
Epoch 11 => Loss: 2.3029685020446777
Epoch 12 => Loss: 1.7779242992401123
Epoch 13 => Loss: 1.2055810689926147
Epoch 14 => Loss: 0.6818041205406189
Epoch 15 => Loss: 0.5160887837409973


In [ ]:
weights

tensor([ 0.3738,  0.7640, -0.3614,  0.4764,  0.3168,  0.5498, -0.5607,  0.7137,
        -0.0060, -0.0221,  0.3816,  0.2074,  0.8004,  0.4415,  0.1404, -0.0430,
         0.5772,  0.5365,  0.0847,  0.5016,  0.2134, -0.5687, -0.4899, -0.2296],
       requires_grad=True)

### Saving model weights for future inference

In [ ]:
torch.save(weights, "linear_model_weights.pt")

### Test Accuracy

In [ ]:
test_preds = calculate_preds(weights, test_features)
test_preds[:5]

tensor([ 0.5129, -0.4021, -0.5087, -0.8819, -0.4508], grad_fn=<SliceBackward0>)

In [ ]:
results = test_labels == (test_preds > 0.5).int()
results[:5]

tensor([False,  True,  True,  True, False])

In [ ]:
results.float().mean()

tensor(0.7264)

In [ ]:
def accuracy(weights, features):
  preds = calculate_preds(weights, features)
  results = test_labels == (preds > 0.5).int()
  return results.float().mean()

In [ ]:
trained_weights = torch.load("linear_model_weights.pt")

In [ ]:
accuracy(trained_weights, test_features)

tensor(0.7264)

*Our linear model's accuracy* **72.64%**